In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from tqdm import tqdm
from joblib import Parallel, delayed
from math import floor, ceil
import os

In [2]:
DATA_DIR = '..\\data\\raw'
INTERIM_DIR = '..\\data\\interim'

### Initial processing of files

In [3]:
df = pd.read_csv(os.path.join(DATA_DIR, 'title.basics.tsv'), sep='\t')
df.head()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [4]:
# Listing all possible titleTypes

df['titleType'].unique()

array(['short', 'movie', 'tvShort', 'tvMovie', 'tvSeries', 'tvEpisode',
       'tvMiniSeries', 'tvSpecial', 'video', 'videoGame', 'episode'],
      dtype=object)

## Strategy

- Framing the problem as predicting movie ratings for 'movie' and 'tvMovie' 
- Convert the 'startYear' field to numeric values and drop the 'endYear' field as it is irrelevant for movies
- Reducing the scope of the problem by filtering movies to the ones that were released after 2010 

#### Treating missing values for startYear:

- Currently, we are setting the startYear as 0 for missing values. However, a better approach would be to impute the year of the movie from other tables. For eg. we have the data on which actors/directors/crew worked on the movie from title.crew and title.principals files. We could collect the crew names and find them in name.basics file. Then we could compute a time when all of these people were alive (a time between [max of their birthYear + some constant] and [min of their deathYear]) and choose a year from that time.  

In [5]:
# del df['endYear']
df['startYear'] = df['startYear'].apply(lambda x: int(x) if (x!='\\N') else 0)

movies = df.loc[((df['titleType']=='movie') | (df['titleType']=='tvMovie')) & (df['startYear']>2010)]
movies.reset_index(drop=True, inplace=True)
movies.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0011216,movie,Spanish Fiesta,La fête espagnole,0,2019,\N,67,Drama
1,tt0011801,movie,Tötet nicht mehr,Tötet nicht mehr,0,2019,\N,\N,"Action,Crime"
2,tt0016906,movie,Frivolinas,Frivolinas,0,2014,\N,80,"Comedy,Musical"
3,tt0019996,movie,Hongxia,Hongxia,0,2011,\N,94,\N
4,tt0062336,movie,El Tango del Viudo y Su Espejo Deformante,El Tango del Viudo y Su Espejo Deformante,0,2020,\N,70,Drama


In [6]:
len(movies)

201093

### Dealing with Null Values

In [7]:
def process_null_values(df):
    '''
    Inputs a dataframe and handles the null values.
    Since data uses a non-standard identifier ('\\N') for null,
    we convert all these values element-wise to np.NaN for pandas processing
    '''
    return df.applymap(lambda x: np.NaN if (x=='\\N') else x)

In [8]:
## Dataset has '\\N' in place of NaN therefore converting it to NaN
df = process_null_values(df)
movies = process_null_values(movies)
movies

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0011216,movie,Spanish Fiesta,La fête espagnole,0,2019,NaN,67,Drama
1,tt0011801,movie,Tötet nicht mehr,Tötet nicht mehr,0,2019,NaN,NaN,"Action,Crime"
2,tt0016906,movie,Frivolinas,Frivolinas,0,2014,NaN,80,"Comedy,Musical"
3,tt0019996,movie,Hongxia,Hongxia,0,2011,NaN,94,NaN
4,tt0062336,movie,El Tango del Viudo y Su Espejo Deformante,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
...,...,...,...,...,...,...,...,...,...
201088,tt9916622,movie,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,0,2015,NaN,57,Documentary
201089,tt9916692,tvMovie,Teatroteka: Czlowiek bez twarzy,Teatroteka: Czlowiek bez twarzy,0,2015,NaN,66,Drama
201090,tt9916706,movie,Dankyavar Danka,Dankyavar Danka,0,2013,NaN,NaN,Comedy
201091,tt9916730,movie,6 Gunn,6 Gunn,0,2017,NaN,116,NaN


In [9]:
movies.isnull().sum()

tconst                 0
titleType              0
primaryTitle           0
originalTitle          0
isAdult                0
startYear              0
endYear           201093
runtimeMinutes     51352
genres              7605
dtype: int64

#### Treating missing values for genres:

- For treating missing values of genres, the following approaches come to mind:  
  
  
  - **Calculating genres from crew**: (Implementation later in the notebook)
    1. Using the crew and principals file, we could get the people associated (nconsts) with the title
    2. Using the 'knownForTitles' field in the name.basics file, we could get a list of all titles (tconsts) these people are known for.
    3. Using the title.basics file, we now can get the genres associated with the above titles and generate a dictionary of counts of genres that appear in these titles. 
    4. We then take the genres with the top 3 counts (since genres can have at most 3 strings) in that dictionary and associate it with the title.
    
    **NOTE:** We use this approach only when the genres field is empty, we do not do it for records where there are less than 3 strings as it is not logical to do so in my opinion. 
  
      
  - **Using Word embeddings with text of reviews**:  
    1. If we had the text reviews of the movies available, we could use a word embedding to associate the reviews with the available genres. In particular, we could use the words occuring in the reviews and calculate their similarity distance (cosine distance) with each of the genres that we find in our dataset. We thus get a dictionary of genres and their similarity with the reviews. We could again take the genres with the top 3 values in this vector and use those genres. 

In [10]:
# Collecting all possible movie genres by splitting the genres field by commas and adding them to a set

genres = set()
df['genres'].apply(lambda x: genres.update(x.split(',')) if isinstance(x, str) else False)
genres

{'Action',
 'Adult',
 'Adventure',
 'Animation',
 'Biography',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Film-Noir',
 'Game-Show',
 'History',
 'Horror',
 'Music',
 'Musical',
 'Mystery',
 'News',
 'Reality-TV',
 'Romance',
 'Sci-Fi',
 'Short',
 'Sport',
 'Talk-Show',
 'Thriller',
 'War',
 'Western'}

In [11]:
len(genres)

28

#### Replacing NaN values in runtimeMinutes by mean of the column

In [12]:
mean_runtime = int(movies['runtimeMinutes'].dropna().astype(int).mean())

movies['runtimeMinutes'] = movies['runtimeMinutes'].apply(lambda x: mean_runtime if (pd.isna(x)) else x)

### Processing the names.basics, title.crew, title.principals files for handling the genre missing values

In [13]:
names = pd.read_csv(os.path.join(DATA_DIR, 'name.basics.tsv'), sep='\t')
names = process_null_values(names)
names

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0050419,tt0031983,tt0053137,tt0072308"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0037382,tt0038355,tt0117057,tt0071877"
2,nm0000003,Brigitte Bardot,1934,NaN,"actress,soundtrack,music_department","tt0057345,tt0054452,tt0049189,tt0056404"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0077975,tt0078723,tt0072562,tt0080455"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050986,tt0060827,tt0050976,tt0069467"
...,...,...,...,...,...,...
10852297,nm9993714,Romeo del Rosario,NaN,NaN,"animation_department,art_department",tt2455546
10852298,nm9993716,Essias Loberg,NaN,NaN,NaN,NaN
10852299,nm9993717,Harikrishnan Rajan,NaN,NaN,cinematographer,tt8736744
10852300,nm9993718,Aayush Nair,NaN,NaN,cinematographer,NaN


In [14]:
names.isnull().sum()

nconst                      0
primaryName                 0
birthYear            10329355
deathYear            10664496
primaryProfession     2294306
knownForTitles        1999277
dtype: int64

In [15]:
# Getting the distribution of the counts of knownforTitles across the dataset

names['knownForTitles'].apply(lambda x: len(x.split(',')) if (not pd.isna(x)) else 0).describe()

count    1.085230e+07
mean     1.562484e+00
std      1.341015e+00
min      0.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      2.000000e+00
max      6.000000e+00
Name: knownForTitles, dtype: float64

In [16]:
title_crew = pd.read_csv(os.path.join(DATA_DIR, 'title.crew.tsv'), sep='\t')
title_crew = process_null_values(title_crew)
title_crew.head()

,tconst,directors,writers
0,tt0000001,nm0005690,NaN
1,tt0000002,nm0721526,NaN
2,tt0000003,nm0721526,NaN
3,tt0000004,nm0721526,NaN
4,tt0000005,nm0005690,NaN


In [17]:
len(title_crew)

7803001

In [18]:
title_principals = pd.read_csv(os.path.join(DATA_DIR, 'title.principals.tsv'), sep='\t')
title_principals = process_null_values(title_principals)
title_principals.head()

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,NaN,"[""Self""]"
1,tt0000001,2,nm0005690,director,NaN,NaN
2,tt0000001,3,nm0374658,cinematographer,director of photography,NaN
3,tt0000002,1,nm0721526,director,NaN,NaN
4,tt0000002,2,nm1335271,composer,NaN,NaN


In [19]:
len(title_principals)

44119644

#### We do not need all the records from the title.crew and title.principals files, we just need the ones associated with the movies we are considering

In [20]:
# Filtering the crew and principals dataframes to include only relevant titles

# Add the titles we are considering to a set and then filter the dataframes
considered_titles = set()
considered_titles.update(movies['tconst'].values)

title_principals = title_principals[title_principals['tconst'].isin(considered_titles)]
title_crew = title_crew[title_crew['tconst'].isin(considered_titles)]

print("Length of processed title_crew: {}".format(len(title_crew)))
print("Length of processed title_principals: {}".format(len(title_principals)))

Length of processed title_crew: 201093
Length of processed title_principals: 1415023


In [21]:
# Saving files as csvs to Interim data folder

title_crew.to_csv(os.path.join(INTERIM_DIR, 'title.crew.csv'), index=False)
title_principals.to_csv(os.path.join(INTERIM_DIR, 'title.principals.csv'), index=False)

#### Filtering the names.basics dataframe for including only the names associated with considered movies for faster processing

**NOTE**: This filtering is only for saving time in prototyping further in the pipeline as I plan to calculate features for each person. In production, I would retain the entire name list and calculate the features on them as well in case a movie has a crew member who hasnt worked in a movie before (eg. an actor who only did TV series up until now).

In [22]:
considered_name_ids = set()

def split_nameIDs(nconst_str):
    '''
    Split the nconst string by commas and filter out empty strings
    '''
    return [nameID for nameID in nconst_str.split(',') if nameID!='']

# Adding all directors
title_crew['directors'].dropna().apply(lambda x: considered_name_ids.update(split_nameIDs(x)))

# Adding all writers
title_crew['writers'].dropna().apply(lambda x: considered_name_ids.update(split_nameIDs(x)))

# Adding principal crew members
considered_name_ids.update(title_principals['nconst'].dropna().values)

print(len(considered_name_ids))

777731


In [23]:
names = names[names['nconst'].isin(considered_name_ids)]
names.reset_index(drop=True, inplace=True)
names

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0050419,tt0031983,tt0053137,tt0072308"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0037382,tt0038355,tt0117057,tt0071877"
2,nm0000003,Brigitte Bardot,1934,NaN,"actress,soundtrack,music_department","tt0057345,tt0054452,tt0049189,tt0056404"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0077975,tt0078723,tt0072562,tt0080455"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050986,tt0060827,tt0050976,tt0069467"
...,...,...,...,...,...,...
777273,nm9993650,Marcin Balcerak,NaN,NaN,actor,tt8739208
777274,nm9993680,Christopher-Lawson Palmer,NaN,NaN,actor,"tt10427366,tt10979852,tt8295580"
777275,nm9993690,David Jewell,NaN,NaN,NaN,tt7888884
777276,nm9993691,Ursula Gehrmann,NaN,NaN,NaN,tt7888884


In [24]:
names.to_csv(os.path.join(INTERIM_DIR, 'names.basics.csv'))

### Pipeline for processing NaN values in genres

In [25]:
genre_null_index = movies[movies['genres'].isna()].index

In [26]:
movies.iloc[list(genre_null_index)]

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
3,tt0019996,movie,Hongxia,Hongxia,0,2011,NaN,94,NaN
17,tt0137818,movie,Housesitter: The Night They Saved Siegfried's ...,Housesitter: The Night They Saved Siegfried's ...,0,2018,NaN,95,NaN
22,tt0167774,movie,At Uluru,At Uluru,0,2021,NaN,80,NaN
26,tt0184221,movie,Appunti per un film sul Jazz,Appunti per un film sul Jazz,0,2015,NaN,87,NaN
27,tt0187902,movie,How Huang Fei-hong Rescued the Orphan from the...,How Huang Fei-hong Rescued the Orphan from the...,0,2011,NaN,87,NaN
...,...,...,...,...,...,...,...,...,...
200987,tt9907032,movie,Footloose in London II: 2 Undiscovered and Unu...,Footloose in London II: 2 Undiscovered and Unu...,0,2018,NaN,124,NaN
200988,tt9907396,movie,Footloose in the Cotswolds - Part 1,Footloose in the Cotswolds - Part 1,0,2016,NaN,118,NaN
200989,tt9907608,movie,Footloose in the Cotswolds - Part 2,Footloose in the Cotswolds - Part 2,0,2016,NaN,102,NaN
201033,tt9910922,movie,Doctor Who Augmented Reality: Times Magazine,Doctor Who Augmented Reality: Times Magazine,0,2013,NaN,87,NaN


In [27]:
def get_genres(tconst, debug=False):
    '''
    Given a tconst that has no genre associated with it, 
    return a string with 3 genres comma-separated estimated
    using the top 3 genres its crew is involved in.
    '''
    
    title_crew = pd.read_csv(os.path.join(INTERIM_DIR, 'title.crew.csv'), usecols=['tconst', 'directors', 'writers'])
    title_principals = pd.read_csv(os.path.join(INTERIM_DIR, 'title.principals.csv'), usecols=['tconst', 'nconst'])
    names = pd.read_csv(os.path.join(INTERIM_DIR, 'names.basics.csv'), usecols=['nconst', 'knownForTitles'])
    
    people_involved = set()
    
    # Adding actors associated with the movie to the set
    people_involved.update(title_principals[title_principals['tconst']==tconst]['nconst'].values)
    
    # Adding writers and directors
    crew = title_crew[title_crew['tconst']==tconst]
    
    if not pd.isna(crew['writers'].iloc[0]):
        people_involved.update(split_nameIDs(crew['writers'].iloc[0]))
    
    if not pd.isna(crew['directors'].iloc[0]):
        people_involved.update(split_nameIDs(crew['directors'].iloc[0]))
    
    # Choose the people involved in the names list
    people_involved_df = names[names['nconst'].isin(people_involved)]
    
    if debug:
        print(people_involved_df)
    
    # We just need the titles that they are collectively known for
    people_titles = people_involved_df['knownForTitles'].dropna()
    
    people_titles_set = set()
    people_titles.apply(lambda x: people_titles_set.update(split_nameIDs(x)))
    
    if debug:
        print(people_titles_set)
    
    # Filter these records from title.basics file to get all the genres associated with them
    genres = df[df['tconst'].isin(people_titles_set)]['genres'].dropna()
    genres_list = []
    genres.apply(lambda x: genres_list.extend(split_nameIDs(x)))
    genres = Counter(genres_list)
    
    if debug:
        print(genres)
        print(genres.most_common(3))
    
    genres = [genre for (genre, count) in genres.most_common(3)]
    
    return ','.join(genres)

#### Trying out the get_genres() function

In [28]:
get_genres('tt0019996', debug=True)

          nconst                             knownForTitles
28142  nm0266662    tt0057713,tt5348132,tt0355037,tt0206738
81551  nm0920761   tt0205557,tt11439364,tt0205573,tt0349368
83451  nm0946300  tt0369552,tt10452668,tt13770570,tt0348490
84289  nm0955817    tt0192690,tt1851091,tt0453321,tt0198850
{'tt0198850', 'tt0205573', 'tt10452668', 'tt0348490', 'tt0192690', 'tt5348132', 'tt1851091', 'tt0369552', 'tt0206738', 'tt11439364', 'tt0205557', 'tt0349368', 'tt13770570', 'tt0453321', 'tt0057713', 'tt0355037'}
Counter({'Drama': 12, 'Action': 2, 'Romance': 2, 'Thriller': 1, 'History': 1, 'Mystery': 1, 'War': 1, 'Musical': 1})
[('Drama', 12), ('Action', 2), ('Romance', 2)]


'Drama,Action,Romance'

#### Parallel implementation using joblib

In [29]:
import itertools 
def get_genres_parallel(tconst_list, verbose=30 ,debug=False):
    
    num_cores = os.cpu_count()
    items_per_core = ceil(len(tconst_list)/num_cores)
    
    list_per_core = [tconst_list[i*items_per_core: (i+1)*items_per_core]
                     for i in range(num_cores)]
    
    if debug:
        print([len(list_) for list_ in list_per_core])

    title_crew = pd.read_csv(os.path.join(INTERIM_DIR, 'title.crew.csv'), usecols=['tconst', 'directors', 'writers'])
    title_principals = pd.read_csv(os.path.join(INTERIM_DIR, 'title.principals.csv'), usecols=['tconst', 'nconst'])
    names = pd.read_csv(os.path.join(INTERIM_DIR, 'names.basics.csv'), usecols=['nconst', 'knownForTitles'])
    
    genres_mapping = Parallel(n_jobs=-1,verbose=verbose, max_nbytes=None)(delayed(get_genres_core)(tc_list, title_crew.copy(),title_principals.copy(),names.copy())
                                    for tc_list in list_per_core)
    result = []
    [result.extend(list_) for list_ in genres_mapping if list_!=[]]
    return result

In [30]:
def get_genres_core(tconst_list, title_crew, title_principals, names, debug=False):
    
    output = []
    
    for (ind, tconst) in tconst_list:
        
        people_involved = set()
    
        # Adding actors associated with the movie to the set
        people_involved.update(title_principals[title_principals['tconst']==tconst]['nconst'].values)

        # Adding writers and directors
        crew = title_crew[title_crew['tconst']==tconst]

        if not pd.isna(crew['writers'].iloc[0]):
            people_involved.update(split_nameIDs(crew['writers'].iloc[0]))

        if not pd.isna(crew['directors'].iloc[0]):
            people_involved.update(split_nameIDs(crew['directors'].iloc[0]))

        # Choose the people involved in the names list
        people_involved_df = names[names['nconst'].isin(people_involved)]

        if debug:
            print(people_involved_df)

        # We just need the titles that they are collectively known for
        people_titles = people_involved_df['knownForTitles'].dropna()

        people_titles_set = set()
        people_titles.apply(lambda x: people_titles_set.update(split_nameIDs(x)))

        if debug:
            print(people_titles_set)

        # Filter these records from title.basics file to get all the genres associated with them
        genres = df[df['tconst'].isin(people_titles_set)]['genres'].dropna()
        genres_list = []
        genres.apply(lambda x: genres_list.extend(split_nameIDs(x)))
        genres = Counter(genres_list)

        if debug:
            print(genres)
            print(genres.most_common(3))

        genres = [genre for (genre, count) in genres.most_common(3)]
        if len(genres)==0:
            genres = ['Drama']
        output.append((ind, ','.join(genres)))
    
    return output

#### Demonstrating on 100 records 

In [31]:
# Get the indices of the rows with no genres
no_gen_ind = movies.index[movies['genres'].isna()].tolist()

tconst_list = list(zip(no_gen_ind, movies['tconst'][no_gen_ind]))
# print(tconst_list)

genres_mapping = get_genres_parallel(tconst_list[0:100], debug=True)

[9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 1]


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done   2 out of  12 | elapsed:  1.8min remaining:  8.9min
[Parallel(n_jobs=-1)]: Done   3 out of  12 | elapsed:  2.0min remaining:  5.9min
[Parallel(n_jobs=-1)]: Done   4 out of  12 | elapsed:  2.1min remaining:  4.3min
[Parallel(n_jobs=-1)]: Done   5 out of  12 | elapsed:  2.3min remaining:  3.2min
[Parallel(n_jobs=-1)]: Done   6 out of  12 | elapsed:  2.5min remaining:  2.5min
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:  2.7min remaining:  1.9min
[Parallel(n_jobs=-1)]: Done   8 out of  12 | elapsed:  2.9min remaining:  1.4min
[Parallel(n_jobs=-1)]: Done   9 out of  12 | elapsed:  3.0min remaining:  1.0min
[Parallel(n_jobs=-1)]: Done  10 out of  12 | elapsed:  3.2min remaining:   38.4s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  3.3min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapse

In [32]:
index, genres = zip(*genres_mapping)

print(list(genres)[:10])

['Drama,Action,Romance', 'Drama,Comedy,Horror', 'Drama', 'Drama,Romance,Documentary', 'Action,Drama,Romance', 'Drama', 'Action,Adventure,Drama', 'Comedy,Family,Drama', 'Drama,Action,Comedy', 'Drama,Action,Comedy']


In [33]:
movies.loc[index, "genres"] = genres

In [34]:
movies['genres'].iloc[list(index)]

3            Drama,Action,Romance
17            Drama,Comedy,Horror
22                          Drama
26      Drama,Romance,Documentary
27           Action,Drama,Romance
                  ...            
2980       Drama,Action,Adventure
3027                        Drama
3030                        Drama
3031                        Drama
3085     Documentary,History,News
Name: genres, Length: 100, dtype: object

In [35]:
movies.to_csv(os.path.join(INTERIM_DIR, 'movies.csv'))

#### Setting genres to 'Drama' in the interest of time for remaining records missing the same

In [36]:
no_gen_ind = movies.index[movies['genres'].isna()].tolist()

movies.loc[no_gen_ind, "genres"] = 'Drama'
movies.iloc[no_gen_ind]

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
3105,tt10243564,tvMovie,François-Xavier Demaison,François-Xavier Demaison,0,2019,NaN,92,Drama
3109,tt10243644,movie,Nuevo Día,Nuevo Día,0,2019,NaN,87,Drama
3176,tt10250836,tvMovie,Paris 1900: The City of Lights,Une si Belle Époque! La France d'avant 1914,0,2019,NaN,103,Drama
3236,tt10256816,movie,Rrëfime për Kosovën,Rrëfime për Kosovën,0,2014,NaN,45,Drama
3241,tt10256990,movie,Leónidas,Leónidas,0,2019,NaN,87,Drama
...,...,...,...,...,...,...,...,...,...
200987,tt9907032,movie,Footloose in London II: 2 Undiscovered and Unu...,Footloose in London II: 2 Undiscovered and Unu...,0,2018,NaN,124,Drama
200988,tt9907396,movie,Footloose in the Cotswolds - Part 1,Footloose in the Cotswolds - Part 1,0,2016,NaN,118,Drama
200989,tt9907608,movie,Footloose in the Cotswolds - Part 2,Footloose in the Cotswolds - Part 2,0,2016,NaN,102,Drama
201033,tt9910922,movie,Doctor Who Augmented Reality: Times Magazine,Doctor Who Augmented Reality: Times Magazine,0,2013,NaN,87,Drama


In [37]:
movies

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0011216,movie,Spanish Fiesta,La fête espagnole,0,2019,NaN,67,Drama
1,tt0011801,movie,Tötet nicht mehr,Tötet nicht mehr,0,2019,NaN,87,"Action,Crime"
2,tt0016906,movie,Frivolinas,Frivolinas,0,2014,NaN,80,"Comedy,Musical"
3,tt0019996,movie,Hongxia,Hongxia,0,2011,NaN,94,"Drama,Action,Romance"
4,tt0062336,movie,El Tango del Viudo y Su Espejo Deformante,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
...,...,...,...,...,...,...,...,...,...
201088,tt9916622,movie,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,0,2015,NaN,57,Documentary
201089,tt9916692,tvMovie,Teatroteka: Czlowiek bez twarzy,Teatroteka: Czlowiek bez twarzy,0,2015,NaN,66,Drama
201090,tt9916706,movie,Dankyavar Danka,Dankyavar Danka,0,2013,NaN,87,Comedy
201091,tt9916730,movie,6 Gunn,6 Gunn,0,2017,NaN,116,Drama


In [46]:
movies.isnull().sum()

tconst                 0
titleType              0
primaryTitle           0
originalTitle          0
isAdult                0
startYear              0
endYear           201093
runtimeMinutes         0
genres                 0
dtype: int64

In [38]:
movies.to_csv(os.path.join(INTERIM_DIR, 'movies.csv'))

In [39]:
title_ratings = pd.read_csv(os.path.join(DATA_DIR, 'title.ratings.tsv'), sep='\t')
title_ratings = process_null_values(title_ratings)
title_ratings

,tconst,averageRating,numVotes
0,tt0000001,5.6,1696
1,tt0000002,6.0,210
2,tt0000003,6.5,1448
3,tt0000004,6.1,122
4,tt0000005,6.1,2245
...,...,...,...
1139443,tt9916580,7.7,6
1139444,tt9916690,6.6,5
1139445,tt9916720,6.4,77
1139446,tt9916766,6.9,16


In [40]:
title_ratings.isnull().sum()

tconst           0
averageRating    0
numVotes         0
dtype: int64

In [41]:
title_ratings.to_csv(os.path.join(INTERIM_DIR, 'title_ratings.csv'))

In [42]:
title_akas = pd.read_csv(os.path.join(DATA_DIR, 'title.akas.tsv'), sep='\t')
title_akas = process_null_values(title_akas)
title_akas

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,NaN,imdbDisplay,NaN,0
1,tt0000001,2,Carmencita,DE,NaN,NaN,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,NaN,imdbDisplay,NaN,0
3,tt0000001,4,Καρμενσίτα,GR,NaN,imdbDisplay,NaN,0
4,tt0000001,5,Карменсита,RU,NaN,imdbDisplay,NaN,0
...,...,...,...,...,...,...,...,...
25897377,tt9916852,4,エピソード #3.20,JP,ja,NaN,NaN,0
25897378,tt9916852,5,Episódio #3.20,PT,pt,NaN,NaN,0
25897379,tt9916852,6,Episodio #3.20,IT,it,NaN,NaN,0
25897380,tt9916852,7,एपिसोड #3.20,IN,hi,NaN,NaN,0


In [43]:
title_akas.isnull().sum()

titleId                   0
ordering                  0
title                     3
region               617749
language            4377488
types              23865156
attributes         25678193
isOriginalTitle        2189
dtype: int64

In [44]:
title_akas = title_akas[title_akas['titleId'].isin(considered_titles)][['titleId', 'ordering']]
title_akas

,titleId,ordering
28907,tt0011216,1
28908,tt0011216,2
28909,tt0011216,3
28910,tt0011216,4
28911,tt0011216,5
...,...,...
25896977,tt9916622,1
25897055,tt9916692,1
25897059,tt9916706,1
25897110,tt9916730,1


In [45]:
title_akas.to_csv(os.path.join(INTERIM_DIR, 'title.akas.csv'))